In [3]:
# import libraries
import pandas as pd
import json
import os
import numpy as np
import pickle
import json
import scipy
import plotly.io as pio
import plotly.express as px
from sklearn.preprocessing import OneHotEncoder
# pandas profile 
import pandas_profiling as pp
from IPython.display import display, HTML

# ignore warnings jupyter notebook
import warnings
warnings.filterwarnings('ignore')

# --------- Functions ----------

In [4]:
# function to check if the data is complete
def check_hours(df):
    """"Function to double check if hours are complete, delete hours with more than 3 zeroes"""

    minutes = ['00:00', '05:00', '10:00', '15:00', '20:00', '25:00', '30:00', '35:00', '40:00', '45:00', '50:00','55:00']
    for date in df['timestamp'].dt.date.unique():
        current_day = df[df['timestamp'].dt.date == date]
        for hour in current_day['timestamp'].dt.hour.unique():
            current_hour = current_day[current_day['timestamp'].dt.hour == hour]
            if (len(current_hour) != 12): # 12 because we have 12, 5 minutes intervals
                df.drop(current_hour.index, inplace=True)
            #Q: should we delete hours with more than 3 zeroes? What would be the impact? A lot of zeroes in the cars column means that the data is sparse ?
            # try:
            #     if current_hour['cars'].value_counts()[0.0] > 3: # delete hours with more than 3 zeroes in the cars column 
            #         df.drop(current_hour.index, inplace=True)
            # except Exception as e:
            #     continue
    return df

In [5]:
# For the Hague Dataset
start_date = '2018-01-01 00:00:00'
end_date = '2020-04-01 23:55:00'
# generate all the dates between start and end date with 5 minutes interval
base_date_df = pd.DataFrame(pd.date_range(start_date, end_date, freq='5min'), columns=['timestamp'])

# read folder with all csv files and create one df from it (one per intersection)
def read_folder(current_intersection, configs, trac, direc):
    """Function to read all csv files (which is one per month) in the path and create one df from it."""

    print("Starting intersection: " + str(current_intersection))  # note which intersection its working on
    path = os.path.join(configs['data_folder'],current_intersection)  # define path to intersection folder (where all csv files are)
    print(path)

    df = pd.DataFrame(columns=['timestamp', 'cars'])  # create df to save everything in
    # loop through all files that end with csv:
    for file in os.listdir(path):  # read all files:
        if file.endswith(".csv"):  # for all csv files in the folder

            current_month = pd.read_csv(os.path.join(path,file), delimiter=";") # read csv file
            cols = configs["trajectories"][trac][direc][current_intersection] + [current_intersection]  # get sensors defined in config file + intersection name for dates (check csv files))
            current_month = current_month[cols]  # only keep interesting columns
            # some cleaning:
            current_month = current_month[:-1]  # last row is totals
            current_month = current_month.fillna(0)  # fill NA values with 0

            # remove sensor errors:
            for sensor in configs['trajectories'][trac][direc][current_intersection]:
                current_month[sensor] = current_month[sensor].apply(lambda x: x if x <= 600 else 0)  # remove sensor errors
                current_month[sensor] = current_month[sensor].loc[current_month[sensor].shift(4) != current_month[sensor]] # remove sensor errors 
            
            current_month[configs['trajectories'][trac][direc][current_intersection]] = \
                current_month[configs['trajectories'][trac][direc][current_intersection]].clip(0,400)  # clip values between 0 and 400

            # sum all sensors: 
            current_month['cars'] = current_month[configs['trajectories'][trac][direc][current_intersection]].sum(axis=1)  # sum of all interesting columns
            current_month = current_month[[current_intersection, "cars"]]  # only keep name and total amount of cars
            current_month.columns = ['timestamp', 'cars']  # rename to timestamp for general format
            
            # add to base df:
            df = pd.concat([df, current_month])

    df['timestamp'] = pd.to_datetime(df['timestamp'])  # format as dt
    df['cars'] = df['cars'].clip(0, len(configs['trajectories'][trac][direc][current_intersection] * 150))  # no intersection could be able to process sensors*150 cars
    df = df.sort_values(by='timestamp')  #sort by timestamp
    df = df.dropna()  # extra check to drop na values
    df = df.loc[(df['timestamp'] > start_date) & (df['timestamp'] < end_date)]  # delete faulty datapoints outside scope
    
    # fill missing values and removing duplicates if any:
    df = pd.merge(base_date_df, df, on='timestamp', how='left') # left join base_date_df with raw_data[intersection] on timestamp
    print("missing value persentage is - ",df['cars'].isnull().sum()/len(df))  # check percentage of missing values
    df['cars'] = df['cars'].interpolate(method='linear', limit_direction='both')  # interpolate missing values
    df = df.drop_duplicates(subset=['timestamp'])  # drop duplicates
    df = df.reset_index(drop=True)
    # df = check_hours(df)  # try this afterwards
    return df



In [6]:
# function to save the processed data for GNN training in h5 format
def save_GNN_processed_data(raw_data,save_path):
    # 1. from raw data dictionary create a dataframe with the values of the dictionary
    # 2. from values in the dictionary change column name 'car' to key name 
    # 3. convert timestamp to datetime64
    # 4. join df's on timestamp

    first_intersection = list(raw_data.keys())[0]
    base_df = raw_data[first_intersection]['timestamp']
    base_df = pd.DataFrame(base_df)
    # base_df['timestamp'] = np.datetime64(base_df['timestamp'])

    for intersection in raw_data:
        df = pd.DataFrame(raw_data[intersection])
        df = df.rename(columns={'cars': intersection})
        # df['timestamp'] = np.datetime64(df['timestamp'])
        base_df = pd.merge(base_df, df, on='timestamp', how='inner')


    base_df['timestamp'] = pd.to_datetime(base_df['timestamp'])
    base_df['timestamp'] = np.array(base_df['timestamp'])


    # set timestamp as index and remove the name of the index
    base_df = base_df.set_index('timestamp')
    base_df.index.name = None

    # save the raw data to a h5 file
    base_df.to_hdf(save_path, key='df', mode='w')

In [7]:
# function to aggregate data into fpds per hour, a new probability column is added with the probability of a car passing through the intersection per 5 minutes
def fpd(df, hours=1):
    """Function to aggregate a df of traffic info into one with fpds per window of x hours (this means 12*hours values)"""
    freq = str(hours) + "H"
    aggregate = df.groupby(pd.Grouper(freq=freq, key='timestamp')).sum()  # aggregate by 1 hour
    df = pd.merge(df, aggregate, on='timestamp', how='left')  # merge with normal df
    df = df.fillna(method='ffill')  # fill with previous number
    df.columns = ['timestamp', 'cars', 'total']
    df['cars'][df['cars'] < 0] = 0  # some inconistencies in the data where cars could be negative
    df['total'][df['total'] <= 0] = 1  # some inconsistencies in the data where total cars could be negative, set to 1 to avoid problems
    df['prob'] = df['cars'] / df['total'] # calculate probability
    return df

In [8]:
# weeks 7,
# hours 24,
# [ data, timesteps ]
# data -> example: FPDs (list of 12 points) of all mondays 00:00 to 01:00 for 4 years with the list of probabilities for each FPD
# timesteps -> example: date associated with each FPD of all mondays 00:00 to 01:00 for 4 years
def create_timeslot_array(data,window=12):
    
    """Function to reshape into numpy array shaped like (samples,window); e.g. 120 datapoints/12 (60min/5mins=12) = 10 FPDs.
    This is neccesary to create the bhattacharyya matrices. Misfunctions when an hour in the data has more or fewer than 12 datapoints (happens with double timestamps or missing data)
    Should be fixed by adding better data protection in the read_data function & rerunning the vlogbroker to output raw sensor values."""

    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data['weekday'] = data['timestamp'].apply(lambda x: x.weekday())
    data['hour'] = data['timestamp'].apply(lambda x: x.hour)
    data_array = [] # create empty array
    for i in range(7):
        timeslots = []
        for hour in range(24):
            # try:
            datapoint = data[(data['weekday'] == i) & (data['hour'] == hour)]
            x = np.array(datapoint['prob'])
            x = x.reshape(int(len(x)/window), window)  # data should be complete and divisible by 12, otherwise it fails.
            # ohe_intersection_array = np.repeat(ohe_intersection, x.shape[0], axis=0)
            # hour_array = np.array([((hour+1)/24)]*x.shape[0]).reshape(-1,1)  # add in hour of day
            # week_array = np.array([((i+1)/7)]*x.shape[0]).reshape(-1,1)  # add in weekday
            # x = np.concatenate((x, hour_array, week_array,ohe_intersection_array), axis=1)  # add in normalized hour of day and weekday
            dates = sorted(set(datapoint['timestamp'].apply(lambda x: x.floor(freq='H'))))  # add in hourly timestamp
            timeslots.append([x, dates])
            # except Exception as e:
            #     print("Exception in create_timeslot_array: ", i, hour)
            #     print(e)
        data_array.append(timeslots)
    # output structure: data_array[7weekdays][24hours]; e.g. data_array[0][9] is data for monday mornings 9 am.
    return data_array

In [9]:
# function to save data to pickle file
def save_pickle(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)

In [10]:
# function to load data from pickle file
def load_pickle(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f)
    return data

# -------- MAIN ----------

## ---------------------------- Hague data processing ---------------------------- 

In [ ]:
# # ---------------------------- hague data ----------------------------

# load configs
with open(r"../utils/configs.json", 'r') as f:
        configs = json.load(f)

final_results = {}  # dictionary to store the results

# loop over all trajectories and directions saperately 
for trajectory in configs['trajectories']:

    # define one hot encoder for the intersections
    enc_intersection = OneHotEncoder(handle_unknown='ignore')
    intersections_list = list(configs['trajectories'][trajectory]['North'].keys())
    enc_intersection.fit(np.array(intersections_list).reshape(-1,1))


    final_results[trajectory] = {}
    for direction in configs['trajectories'][trajectory].keys():
        final_results[trajectory][direction] = {}

        # 1. first read the raw data from the pickle files and create a dictionary with the data:
        raw_data = {}
        for intersection in configs['trajectories'][trajectory][direction]:
            raw_data[intersection] = read_folder(intersection, configs, trajectory, direction) # read the raw data from the pickle files and create a dictionary with the data
        save_path = f"../data/hague/processed/GNN_raw_data_{direction}_{trajectory}.h5" # path to save the processed raw data to a h5 file for GNNs
        save_GNN_processed_data(raw_data, save_path) # save the raw data to a h5 file for GNNs


        # 2. create FPDs from dictionary
        print("Creating FPDs...") 
        fpds = {}
        fpd_hour = 1 # interval in hours to aggregate the data
        for intersection in raw_data:
            fpds[intersection] = fpd(raw_data[intersection], fpd_hour)


        # 3. create timeslot arrays for further processing:
        print("Creating timeslot arrays...")
        featured_fpds = {}  # array with shape (weeks, hours) containing the FPDs for each hour of each week for all days from 2018 to 2022
        window_size = 12 # window size in 5 minute intervals 
        for intersection in fpds:
            # ohe_intersection = enc_intersection.transform(np.array([[intersection]])).toarray() # one hot encode the intersection
            fpds_processed = create_timeslot_array(fpds[intersection], window_size) # create timeslot array
            featured_fpds[intersection] = fpds_processed
        

        # 4. save the featured fpds to a pickle file
        print("Saving featured fpds to pickle file...")
        featured_fpds_save_path = f"../data/hague/processed/featured_fpds_{direction}_{trajectory}.pickle" # path to save the processed fpds to a pickle file
        save_pickle(featured_fpds, featured_fpds_save_path) # save the fpds to a pickle file


        # 5. save in final results dictionary
        final_results[trajectory][direction]['fpds'] = featured_fpds
        final_results[trajectory][direction]['raw'] = raw_data


# save final results to pickle file
final_results_save_path = f"../data/hague/processed/featured_fpds_raw.pickle" # path to save the processed fpds to a pickle file
save_pickle(final_results, "../data/hague/processed/featured_fpds_raw.pickle")

## ---------------------------- METR-LA data processing ---------------------------- 

In [ ]:
final_results = {}
# ---------------------------- METR-LA data ----------------------------
# 1. function to load data from pickle file
raw_data = load_pickle('../data/METR-LA/processed/OWRI_df_format.pickle')

# 2. create FPDs from dictionary 
print("Creating FPDs...")
fpds = {}
fpd_hour = 1 # interval in hours to aggregate the data
for intersection in raw_data:
    fpds[intersection] = fpd(raw_data[intersection], fpd_hour)

# 3. create timeslot arrays for further processing:
print("Creating timeslot arrays...")
featured_fpds = {}  # array with shape (weeks, hours) containing the FPDs for each hour of each week for all days from 2018 to 2022
window_size = 12 # window size in 5 minute intervals 
for intersection in fpds:
    # ohe_intersection = enc_intersection.transform(np.array([[intersection]])).toarray() # one hot encode the intersection
    fpds_processed = create_timeslot_array(fpds[intersection], window_size) # create timeslot array
    featured_fpds[intersection] = fpds_processed


# 4. save the featured fpds to a pickle file
print("Saving featured fpds to pickle file...")
featured_fpds_save_path = f"../data/METR-LA/processed/featured_fpds.pickle" # path to save the processed fpds to a pickle file
save_pickle(featured_fpds, featured_fpds_save_path) # save the fpds to a pickle file


# 5. save in final results dictionary
final_results['fpds'] = featured_fpds
final_results['raw'] = raw_data


# save final results to pickle file
final_results_save_path = f"../data/METR-LA/processed/featured_fpds_raw.pickle" # path to save the processed fpds to a pickle file
save_pickle(final_results, final_results_save_path)

## ---------------------------- PEMS-BAY data processing ---------------------------- 

In [ ]:
final_results = {}
# ---------------------------- METR-LA data ----------------------------
# 1. function to load data from pickle file
raw_data = load_pickle('../data/PEMS-BAY/processed/OWRI_df_format.pickle')

# 2. create FPDs from dictionary 
print("Creating FPDs...")
fpds = {}
fpd_hour = 1 # interval in hours to aggregate the data
for intersection in raw_data:
    fpds[intersection] = fpd(raw_data[intersection], fpd_hour)

# 3. create timeslot arrays for further processing:
print("Creating timeslot arrays...")
featured_fpds = {}  # array with shape (weeks, hours) containing the FPDs for each hour of each week for all days from 2018 to 2022
window_size = 12 # window size in 5 minute intervals 
for intersection in fpds:
    # ohe_intersection = enc_intersection.transform(np.array([[intersection]])).toarray() # one hot encode the intersection
    fpds_processed = create_timeslot_array(fpds[intersection], window_size) # create timeslot array
    featured_fpds[intersection] = fpds_processed


# 4. save the featured fpds to a pickle file
print("Saving featured fpds to pickle file...")
featured_fpds_save_path = f"../data/PEMS-BAY/processed/featured_fpds.pickle" # path to save the processed fpds to a pickle file
save_pickle(featured_fpds, featured_fpds_save_path) # save the fpds to a pickle file


# 5. save in final results dictionary
final_results['fpds'] = featured_fpds
final_results['raw'] = raw_data


# save final results to pickle file
final_results_save_path = f"../data/PEMS-BAY/processed/featured_fpds_raw.pickle" # path to save the processed fpds to a pickle file
save_pickle(final_results, final_results_save_path)

# -----------------------  END -----------------------

# ========== OTHER PRE PROCESSING OVER DATA =========

## ------------- METER-LA and PEMS-BAY FPDs Creation -------------

In [11]:
# function to read the data from the h5 file and convert to dictionary with each column name as key
# read the h5 file from meter-la data and convert to dictionary with each column name as key
def read_h5(df):
    data_dict = {} # create empty dictionary for storing the data
    # make datetime object using year, month, day, hour, minute, second
    # reset index and add timestamp column
    df.index = pd.to_datetime(df.index.year*10000000000 + df.index.month*100000000 + df.index.day*1000000 + df.index.hour*10000 + df.index.minute*100 + df.index.second, format='%Y%m%d%H%M%S')
    df['timestamp'] = df.index
    df.reset_index(drop=True, inplace=True)
    # create dictionary with each column name as key
    for column in df.columns:
        if column == 'timestamp':
            continue
        temp_df = pd.DataFrame()
        temp_df['timestamp'] = df['timestamp']
        temp_df['cars'] = df[column].values
        data_dict[column] = temp_df

    return data_dict

In [13]:
# read data of meter-la 
load_path = '../data/METR-LA/metr-la.h5'
save_path = '../data/METR-LA/processed/OWRI_df_format.pickle'
df = pd.read_hdf(load_path, 'df')
df.head()

,773869,767541,767542,717447,717446,717445,773062,767620,737529,717816,...,772167,769372,774204,769806,717590,717592,717595,772168,718141,769373
2012-03-01 00:00:00,64.375000,67.625000,67.125000,61.500000,66.875000,68.750000,65.125,67.125,59.625000,62.750000,...,45.625000,65.500,64.500000,66.428571,66.875,59.375000,69.000000,59.250000,69.000000,61.875
2012-03-01 00:05:00,62.666667,68.555556,65.444444,62.444444,64.444444,68.111111,65.000,65.000,57.444444,63.333333,...,50.666667,69.875,66.666667,58.555556,62.000,61.111111,64.444444,55.888889,68.444444,62.875
2012-03-01 00:10:00,64.000000,63.750000,60.000000,59.000000,66.500000,66.250000,64.500,64.250,63.875000,65.375000,...,44.125000,69.000,56.500000,59.250000,68.125,62.500000,65.625000,61.375000,69.857143,62.000
2012-03-01 00:15:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000
2012-03-01 00:20:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000


In [ ]:
# convert to dictionary
OWRI_df_format = read_h5(df)

In [ ]:
# save data to pickle file
with open(save_path, 'wb') as f:
    pickle.dump(OWRI_df_format, f)

In [ ]:
# read data of pems-bay
load_path = '../data/PEMS-BAY/pems-bay.h5'
save_path = '../data/PEMS-BAY/processed/OWRI_df_format.pickle'
df = pd.read_hdf(load_path)
df.head()

In [ ]:
# convert to dictionary
OWRI_df_format = read_h5(df)

In [ ]:
# save data to pickle file
with open(save_path, 'wb') as f:
    pickle.dump(OWRI_df_format, f)

## ---------------  Combining hague all trajectory data  ---------------

In [ ]:
# read hague processed data
load_path1 = '../data/hague/processed/GNN_raw_data_North_T1.h5'
df1 = pd.read_hdf(load_path1)
# add "_N" to the column names of the North trajectories
df1.columns = [str(col) + '_N' for col in df1.columns]

load_path2 = '../data/hague/processed/GNN_raw_data_North_T2.h5'
df2 = pd.read_hdf(load_path2)
# add "_N" to the column names of the North trajectories
df2.columns = [str(col) + '_N' for col in df2.columns]

load_path3 = '../data/hague/processed/GNN_raw_data_South_T1.h5'
df3 = pd.read_hdf(load_path3)
# add "_S" to the column names of the South trajectories
df3.columns = [str(col) + '_S' for col in df3.columns]

load_path4 = '../data/hague/processed/GNN_raw_data_South_T2.h5'
df4 = pd.read_hdf(load_path4)
# add "_S" to the column names of the South trajectories
df4.columns = [str(col) + '_S' for col in df4.columns]

In [ ]:
# save the raw data to a h5 file
save_path = '../data/hague/processed/GNN_raw_data_N_T1.h5'
df1 = df1.reindex(sorted(df1.columns), axis=1)
df1.to_hdf(save_path, key='df', mode='w')

save_path = '../data/hague/processed/GNN_raw_data_N_T2.h5'
df2 = df2.reindex(sorted(df2.columns), axis=1)
df2.to_hdf(save_path, key='df', mode='w')

save_path = '../data/hague/processed/GNN_raw_data_S_T1.h5'
df3 = df3.reindex(sorted(df3.columns), axis=1)
df3.to_hdf(save_path, key='df', mode='w')

save_path = '../data/hague/processed/GNN_raw_data_S_T2.h5'
df4 = df4.reindex(sorted(df4.columns), axis=1)
df4.to_hdf(save_path, key='df', mode='w')

In [ ]:
# merge the dataframes on index
df = pd.concat([df1, df2, df3, df4], axis=1)
df = df.dropna()
df = df.reindex(sorted(df.columns), axis=1)
# save in h5 format
df.to_hdf('../data/hague/processed/GNN_raw_data.h5', key='df', mode='w')

# ----------------  Data Exploration ----------------

## A. HAGUE

In [ ]:
# read hague processed data
load_path1 = '../data/hague/processed/GNN_raw_data_North_T1.h5'
df1 = pd.read_hdf(load_path1)
# add "_N" to the column names of the North trajectories
df1.columns = [str(col) + '_N' for col in df1.columns]

load_path2 = '../data/hague/processed/GNN_raw_data_North_T2.h5'
df2 = pd.read_hdf(load_path2)
# add "_N" to the column names of the North trajectories
df2.columns = [str(col) + '_N' for col in df2.columns]

load_path3 = '../data/hague/processed/GNN_raw_data_South_T1.h5'
df3 = pd.read_hdf(load_path3)
# add "_S" to the column names of the South trajectories
df3.columns = [str(col) + '_S' for col in df3.columns]

load_path4 = '../data/hague/processed/GNN_raw_data_South_T2.h5'
df4 = pd.read_hdf(load_path4)
# add "_S" to the column names of the South trajectories
df4.columns = [str(col) + '_S' for col in df4.columns]

In [ ]:
# save the raw data to a h5 file
save_path = '../data/hague/processed/GNN_raw_data_N_T1.h5'
df1 = df1.reindex(sorted(df1.columns), axis=1)
df1.to_hdf(save_path, key='df', mode='w')

save_path = '../data/hague/processed/GNN_raw_data_N_T2.h5'
df2 = df2.reindex(sorted(df2.columns), axis=1)
df2.to_hdf(save_path, key='df', mode='w')

save_path = '../data/hague/processed/GNN_raw_data_S_T1.h5'
df3 = df3.reindex(sorted(df3.columns), axis=1)
df3.to_hdf(save_path, key='df', mode='w')

save_path = '../data/hague/processed/GNN_raw_data_S_T2.h5'
df4 = df4.reindex(sorted(df4.columns), axis=1)
df4.to_hdf(save_path, key='df', mode='w')

In [ ]:
# merge the dataframes on index
df = pd.concat([df1, df2, df3, df4], axis=1)

In [ ]:
# calulate basic stats from the data
print('Number of days: {}'.format(max(df.index)- min(df.index)))
print('total number of samples: {}'.format(len(df.index)))
print('total number of sensors: {}'.format(len(df.columns)))

Number of days: 821 days 23:55:00
total number of samples: 236736
total number of sensors: 48


In [ ]:
# random sample plot
temp = df.iloc[5000:, :]
sample_df = temp.head(12*24*21)
sample_df = sample_df.resample('1H').mean()
fig = px.line(sample_df, x=sample_df.index, y=['K504_N'], width=700, height=450)
fig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Number Of Vehicles')
fig.update_layout(showlegend=False)
fig.show()
pio.write_image(fig, 'plots/rt_speed_hague.png',scale=4, width=700, height=450)

### --------- FPDs plot for 5,15 and 60 min --------- ###

In [ ]:
# select a semple of from december 15 to jaunary 15
temp = df
# plot 5min, 15min and 1 hour intervals smoorhing
# temp = df
sample_df = temp.head(12*24*2)
fig = px.line(sample_df, x=sample_df.index, y=['K504_N'], width=600, height=400)
fig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Number Of Vehicles')
fig.update_layout(showlegend=False)
fig.show()
pio.write_image(fig, 'plots/averaging_5min.png',height=400, width = 600, scale=4)

In [ ]:
sample_df = temp.head(12*24*2)
sample_df = sample_df.resample('15min').mean()
fig = px.line(sample_df, x=sample_df.index, y=['K504_N'], width=600, height=400)
fig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Number Of Vehicles')
fig.update_layout(showlegend=False)
fig.show()
pio.write_image(fig, 'plots/averaging_15min.png',height=400, width = 600, scale=4)

In [ ]:
sample_df = temp.head(12*24*2)
sample_df = sample_df.resample('1H').mean()
fig = px.line(sample_df, x=sample_df.index, y=['K504_N'], width=600, height=400)
fig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Number Of Vehicles')
fig.update_layout(showlegend=False)
fig.show()
pio.write_image(fig, 'plots/averaging_1H.png',height=400, width = 600, scale=4)

### ------- Internode and speed correlations -------

In [ ]:
# drop index of df
df.reset_index(drop=True, inplace=True)
# get all speed values in a list from all columns
speed_list = []
for column in df.columns:
    if column == 'timestamp':
        continue
    speed_list.extend(df[column].values)
corr_df = df.corr()

In [ ]:
# get all values in a list from all columns
corr_list = []
for column in corr_df.columns:
    corr_list.extend(corr_df[column].values)

In [ ]:
plot_metr_df_corr = {
    'pearson correlation': corr_list,
}
plot_metr_df_speed = {
    'Number of vehicles': speed_list,
}
plot_metr_df_corr = pd.DataFrame(plot_metr_df_corr)
plot_metr_df_speed = pd.DataFrame(plot_metr_df_speed)

In [ ]:
# make a histogram of the speed values using plotly
fig = px.histogram(plot_metr_df_speed['Number of vehicles'], title='Vehicle distribution of Hague',nbins=30,histnorm='probability density', width=800, height=500)
fig.update_xaxes(title_text='vehicle count')
fig.show()

In [ ]:
# make a histogram of the speed values using plotly
fig = px.histogram(plot_metr_df_corr['pearson correlation'], title='Pearson Correlation of Hague',nbins=15,histnorm='probability density', width=800, height=500)
fig.update_xaxes(title_text='Pearson Correlation')
fig.show()

## B. METR-LA

In [14]:
# read data of meter-la 
load_path = '../data/METR-LA/metr-la.h5'
save_path = '../data/METR-LA/processed/OWRI_df_format.pickle'
df = pd.read_hdf(load_path, 'df')
df.index = pd.to_datetime(df.index.year*10000000000 + df.index.month*100000000 + df.index.day*1000000 + df.index.hour*10000 + df.index.minute*100 + df.index.second, format='%Y%m%d%H%M%S')
display(HTML(df.head().to_html()))

,773869,767541,767542,717447,717446,717445,773062,767620,737529,717816,765604,767471,716339,773906,765273,716331,771667,716337,769953,769402,769403,769819,769405,716941,717578,716960,717804,767572,767573,773012,773013,764424,769388,716328,717819,769941,760987,718204,718045,769418,768066,772140,773927,760024,774012,774011,767609,769359,760650,716956,769831,761604,717495,716554,773953,767470,716955,764949,773954,767366,769444,773939,774067,769443,767750,767751,767610,773880,764766,717497,717490,717491,717492,717493,765176,717498,717499,765171,718064,718066,765164,769431,769430,717610,767053,767621,772596,772597,767350,767351,716571,773023,767585,773024,717483,718379,717481,717480,717486,764120,772151,718371,717489,717488,717818,718076,718072,767455,767454,761599,717099,773916,716968,769467,717576,717573,717572,717571,717570,764760,718089,769847,717608,767523,716942,718090,769867,717472,717473,759591,764781,765099,762329,716953,716951,767509,765182,769358,772513,716958,718496,769346,773904,718499,764853,761003,717502,759602,717504,763995,717508,765265,773996,773995,717469,717468,764106,717465,764794,717466,717461,717460,717463,717462,769345,716943,772669,717582,717583,717580,716949,717587,772178,717585,716939,768469,764101,767554,773975,773974,717510,717513,717825,767495,767494,717821,717823,717458,717459,769926,764858,717450,717452,717453,759772,717456,771673,772167,769372,774204,769806,717590,717592,717595,772168,718141,769373
2012-03-01 00:00:00,64.375000,67.625000,67.125000,61.500000,66.875000,68.750000,65.125,67.125,59.625000,62.750000,55.500000,66.500000,64.250000,68.500000,60.375000,67.500000,37.75,63.125000,59.750000,62.125000,67.250000,41.250000,54.625000,58.125000,65.125000,64.250000,61.250000,62.750000,66.875000,47.375000,57.000000,67.750000,65.125,66.875000,64.000000,62.000000,64.375,60.750000,60.750,62.375000,64.500000,66.000000,62.000000,64.875000,55.875000,67.750000,63.000000,60.125000,60.250000,62.250000,42.250000,48.250000,65.125,63.500,55.750000,60.500000,51.571429,60.250000,63.875000,59.875000,64.625000,64.000000,52.500000,66.250000,68.250000,68.875000,66.375000,64.500000,67.875000,67.250000,63.125000,66.250000,64.625000,59.125000,68.250000,52.250000,58.125000,61.250000,66.125000,69.375000,69.750000,63.500000,64.25,68.000,56.875000,64.375000,65.250000,65.750000,61.250000,63.625000,67.000000,55.250000,66.875000,65.375,65.125000,57.625000,69.125000,66.500000,62.000000,67.375000,60.125000,65.500000,63.750000,66.250000,65.250000,0.0,0.0,66.875000,63.625000,55.750000,67.375000,65.750000,63.125000,66.000000,68.625,61.500000,67.125000,69.250000,65.375000,68.375000,61.375000,53.750000,65.250000,69.375000,62.125,50.125000,61.750000,66.875000,67.125000,58.50,66.125000,66.375000,65.750000,61.750000,67.250000,65.000000,66.375000,63.875000,67.875000,63.625000,57.625,64.625000,59.750000,63.500000,58.125000,66.750000,66.250000,60.625000,63.750000,69.50,61.250000,53.125,61.375000,63.125000,63.000,63.375,54.750000,65.125000,64.000000,67.375000,67.125000,68.250000,67.625000,69.375000,65.500,55.000,70.000000,68.250000,62.000000,64.500000,58.375000,64.500000,63.625000,63.750000,53.625000,69.875000,64.750000,62.375000,64.750000,52.125000,61.625000,63.000000,67.142857,67.625000,63.875000,63.125000,63.375000,64.125000,66.875000,69.875,67.750000,62.000000,66.750000,57.625000,52.625000,69.000000,43.500000,45.625000,65.500,64.500000,66.428571,66.875,59.375000,69.000000,59.250000,69.000000,61.875
2012-03-01 00:05:00,62.666667,68.555556,65.444444,62.444444,64.444444,68.111111,65.000,65.000,57.444444,63.333333,58.777778,58.444444,66.444444,65.444444,56.111111,65.444444,39.00,63.777778,63.111111,66.111111,65.777778,56.777778,50.777778,56.222222,66.888889,54.444444,67.888889,61.555556,67.555556,46.777778,67.777778,67.888889,68.000,67.444444,66.444444,67.222222,63.000,64.333333,64.000,61.888889,66.777778,64.555556,64.444444,64.333333,66.444444,64.111111,58.888889,65.222222,60.333333,63.111111,52.333333,52.111111,68.000,56.000,60.888889,60.222

In [20]:
# calulate basic stats from the data
print('Number of days: {}'.format(max(df.index)- min(df.index)))
print('total number of samples: {}'.format(len(df.index)))
print('total number of sensors: {}'.format(len(df.columns)))

Number of days: 118 days 23:55:00
total number of samples: 34272
total number of sensors: 207


In [25]:
# plot a sample from the dataset for 3 weeks
temp = df
sample_df = temp.head(12*24*21)
sample_df = sample_df.resample('1H').mean()
fig = px.line(sample_df, x=sample_df.index, y=['773869'], width=700, height=450)
fig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Average Speed (miles/H)')
fig.update_layout(showlegend=False)
pio.write_image(fig, 'plots/rt_speed_metr.png',scale=4, width=700, height=450)
fig.show()

In [35]:
# speed 3 random sensors
sample_df = df.head(12*24)
px.line(sample_df, x=sample_df.index, y=['767542','771667','718076'], title='speed analysis' )

In [36]:
# speed analysis
sample_df = df.head(12*24)
px.line(sample_df, x=sample_df.index, y=['771673','771667','718076'], title='speed analysis' )

In [37]:
# speed analysis
sample_df = df.head(12*24*7)
px.line(sample_df, x=sample_df.index, y=['771673','771667','718076'], title='speed analysis' )

### ------- Internode and speed correlations -------

In [38]:
# drop index of df
df.reset_index(drop=True, inplace=True)
# get all speed values in a list from all columns
speed_list = []
for column in df.columns:
    if column == 'timestamp':
        continue
    speed_list.extend(df[column].values)
corr_df = df.corr()

In [39]:
# get all values in a list from all columns
corr_list = []
for column in corr_df.columns:
    corr_list.extend(corr_df[column].values)

In [40]:
plot_metr_df_corr = {
    'pearson correlation': corr_list,
}
plot_metr_df_speed = {
    'Speed (miles/H)': speed_list,
}
plot_metr_df_corr = pd.DataFrame(plot_metr_df_corr)
plot_metr_df_speed = pd.DataFrame(plot_metr_df_speed)

In [ ]:
# make a histogram of the speed values using plotly
fig = px.histogram(plot_metr_df_speed['Speed (miles/H)'],nbins=15,histnorm='probability density', width=800, height=500)
fig.update_xaxes(title_text='Speed (miles/H)')
fig.update_layout(showlegend=False)
fig.show()
# pio.write_image(fig, 'plots/hague_realtime_weekcompare.png',height=500, width = 900, scale=3)

In [ ]:
# make a histogram of the speed values using plotly
fig = px.histogram(plot_metr_df_corr['pearson correlation'],nbins=15,histnorm='probability density', width=800, height=500)
fig.update_xaxes(title_text='Pearson correlation')
fig.update_layout(showlegend=False)
fig.show()

## C. PEMS-BAY

In [44]:
# read data of meter-la 
load_path = '../data/PEMS-BAY/pems-bay.h5'
save_path = '../data/PEMS-BAY/processed/OWRI_df_format.pickle'
df = pd.read_hdf(load_path)
df.index = pd.to_datetime(df.index.year*10000000000 + df.index.month*100000000 + df.index.day*1000000 + df.index.hour*10000 + df.index.minute*100 + df.index.second, format='%Y%m%d%H%M%S')
df.head()

sensor_id,400001,400017,400030,400040,400045,400052,400057,400059,400065,400069,...,409525,409526,409528,409529,413026,413845,413877,413878,414284,414694
2017-01-01 00:00:00,71.4,67.8,70.5,67.4,68.8,66.6,66.8,68.0,66.8,69.0,...,68.8,67.9,68.8,68.0,69.2,68.9,70.4,68.8,71.1,68.0
2017-01-01 00:05:00,71.6,67.5,70.6,67.5,68.7,66.6,66.8,67.8,66.5,68.2,...,68.4,67.3,68.4,67.6,70.4,68.8,70.1,68.4,70.8,67.4
2017-01-01 00:10:00,71.6,67.6,70.2,67.4,68.7,66.1,66.8,67.8,66.2,67.8,...,68.4,67.4,68.4,67.5,70.2,68.3,69.8,68.4,70.5,67.9
2017-01-01 00:15:00,71.1,67.5,70.3,68.0,68.5,66.7,66.6,67.7,65.9,67.8,...,68.5,67.5,68.5,67.5,70.4,68.7,70.2,68.4,70.8,67.6
2017-01-01 00:20:00,71.7,67.8,70.2,68.1,68.4,66.9,66.1,67.7,66.1,67.8,...,68.5,67.7,68.5,67.4,69.6,69.1,70.0,68.4,71.0,67.9


In [46]:
# calulate basic stats from the data
print('Number of days: {}'.format(max(df.index)- min(df.index)))
print('total number of samples: {}'.format(len(df.index)))
print('total number of sensors: {}'.format(len(df.columns)))

Number of days: 180 days 23:55:00
total number of samples: 52116
total number of sensors: 325


In [45]:
# plot for a random sensor
temp = df.iloc[25000:,:]
sample_df = temp.head(12*24*21)
sample_df = sample_df.resample('1H').mean()
fig = px.line(sample_df, x=sample_df.index, y=[400001], width=700, height=450)
fig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Average Speed (miles/H)')
fig.update_layout(showlegend=False)
fig.show()
pio.write_image(fig, 'plots/rt_speed_pems.png',height=450, width = 700, scale=4)

### ------- Internode and speed correlations -------

In [ ]:
# drop index of df
df.reset_index(drop=True, inplace=True)
# get all speed values in a list from all columns
speed_list = []
for column in df.columns:
    if column == 'timestamp':
        continue
    speed_list.extend(df[column].values)
corr_df = df.corr()

In [ ]:
# get all values in a list from all columns
corr_list = []
for column in corr_df.columns:
    corr_list.extend(corr_df[column].values)

In [ ]:
plot_metr_df_corr = {
    'pearson correlation': corr_list,
}
plot_metr_df_speed = {
    'Speed (miles/H)': speed_list,
}
plot_metr_df_corr = pd.DataFrame(plot_metr_df_corr)
plot_metr_df_speed = pd.DataFrame(plot_metr_df_speed)

In [ ]:
# make a histogram of the speed values using plotly
fig = px.histogram(plot_metr_df_speed['Speed (miles/H)'], title='Speed Distribution of PEMS-BAY',nbins=15,histnorm='probability density', width=800, height=500)
fig.update_xaxes(title_text='Speed (miles/H)')
fig.show()

In [ ]:
# make a histogram of the speed values using plotly
fig = px.histogram(plot_metr_df_corr['pearson correlation'], title='Pearson Correlation of PEMS-BAY',nbins=15,histnorm='probability density', width=800, height=500)
fig.update_xaxes(title_text='pearson correlation')
fig.show()

## --------------------- END ---------------------